In [3]:
import open3d as o3d
import numpy as np
import copy

In [20]:
def creat_crystal(position, direction, H, R):
    # position にて長さ H, （外接円）半径 R, 向きが direction の正六角結晶を作る関数
    # positon = (p1, p2, p3)
    # direction = (d1, d2, d3)
    # direction は原点からのベクトル
    # 原点にて正六角結晶を作る --> 指定した向きに回転させる --> 指定した位置に平行移動させる --> return
    
    # 六角柱と2つのコーンを組み合わせる
    hex_cy = o3d.geometry.TriangleMesh.create_cylinder(radius=R, height=H, resolution=6, split=1)
    
    # 六角コーンを作り、先端の位置を微調整
    hex_con = o3d.geometry.TriangleMesh.create_cone(radius=R, height=H/10, resolution=6, split=1, create_uv_map=False)

    a = np.random.normal(loc = 0, scale = H/50, size = (1, 3))
    vertices2 = np.asarray(hex_con.vertices)
    vertices2[1] = vertices2[1] + a
    hex_con.vertices = o3d.utility.Vector3dVector(vertices2)

    #con_t = copy.deepcopy(hex_con).translate((0, 0, H/2))
    con_t = hex_con.translate((0, 0, H/2))
    
    #con_rev = copy.deepcopy(con_t)
    #R = con_rev.get_rotation_matrix_from_xyz((0, np.pi, 0))
    #con_rev.rotate(R, center=(0, 0, 0))
    
    #底をちょっと太くする
    vertices = np.asarray(hex_cy.vertices)
    new_v = np.concatenate([vertices[0: 8], [i * 1.5 for i in vertices[8: 14]]])
    hex_cy.vertices = o3d.utility.Vector3dVector(new_v)
    
    #vertices2 = np.asarray(con_rev.vertices)
    #new_v2 = [a * 1.2 for a in vertices2]
    #con_rev.vertices = o3d.utility.Vector3dVector(new_v2)
    
    # 組み合わせる
    crystal = con_t + hex_cy # + con_rev
    
    # 回転
    R2 = crystal.get_rotation_matrix_from_xyz(direction)
    crystal.rotate(R2, center=(0, 0, 0))
    
    # 平行移動
    #crystal2 = copy.deepcopy(crystal).translate(position)
    crystal = crystal.translate(position)
    
    #陰をつける(?)
    crystal.compute_vertex_normals()
    
    return crystal

#TEST#####################
position = (0, 0, 0)
direction = (0, 0, 0)
H = 10
R = 1.0
o3d.visualization.draw_geometries([creat_crystal(position, direction, H, R)], mesh_show_wireframe=True)
##########################

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
def grow_crystal(var, num, h, hv, r, rv, Vvar):
    # 平面上に num*num 個の結晶を分散 var で成長させる関数
    # 結晶の長さは N(h, hv)の正規分布に従うとする
    # 結晶の半径は N(h, hv)の正規分布に従うとする
    # 結晶の向きの分散 --> Vvar
    
    # 平面上の点群 xyz をとる（後で creat_crystal関数の position になる
    x = np.linspace(0, 10, num)
    mesh_x, mesh_y = np.meshgrid(x, x)
    z = 0
    xyz = np.zeros((np.size(mesh_x), 3))
    xyz[:, 0] = np.reshape(mesh_x, -1)
    xyz[:, 1] = np.reshape(mesh_y, -1)
    xyz[:, 2] = np.reshape(0, -1)
    
    x_noise = np.random.normal(loc = 0, scale = var, size = num*num)
    y_noise = np.random.normal(loc = 0, scale = var, size = num*num)
    
    xyz[:, 0] = xyz[:, 0] + x_noise
    xyz[:, 1] = xyz[:, 1] + y_noise
    
    # 結晶の集合 --> field
    field = list()
    
    for p in xyz:
        height = np.random.normal(loc = h, scale = hv)
        radius = np.random.normal(loc = r, scale = rv)
        
        d = np.array([np.random.normal(loc = 0, scale = Vvar), np.random.normal(loc = 0, scale = Vvar), np.random.normal(loc = 1, scale = Vvar)])
        
        field.append(creat_crystal(p, d, height, radius))
        
    return field

##########################

# 平面（薄いボックス）--> face
face = o3d.geometry.TriangleMesh.create_box(width=20.0, height=20.0, depth=0.01, create_uv_map=False)
#face2 = copy.deepcopy(face).translate((-50, -50, 0))
face = face.translate((-5, -5, 0))
# 平面に色付ける
face.paint_uniform_color([0.1, 0.1, 0.1])


var = 3
num = 5
h = 5
hv = 1
r = 0.5
rv = 0.1
Vvar = 0.3

model = grow_crystal(var, num, h, hv, r, rv, Vvar)

var2 = 2
num2 = 2
h2 = 10
hv2 = 2
r2 = 1.5
rv2 = 0.3
Vvar2 = 0.3
model2 = grow_crystal(var2, num2, h2, hv2, r2, rv2, Vvar2)

model.append(face)

for i in model2:
    model.append(i)

o3d.visualization.draw_geometries(model)
##########################